In [14]:
import pandas as pd
import os
import re
import glob
import json
import shutil
import html

In [15]:
from bs4 import BeautifulSoup
with open("notion.html", "r") as file:
 html = file.read()
soup = BeautifulSoup(html)

table = soup.find(attrs={'class': 'notion-collection-view-body'}).find(attrs={"class": "notion-collection_view_page-block"})
groups = list()
for group in table.div.children:
    header = group.find_all(attrs={'role': 'button'})
    if len(header) < 2 or not header[1].div:
        continue
    group_name = header[1].div.div.contents[0]
    icons = list()
    for row in list(group.children)[3].find_all(attrs={'class': 'notion-collection-item'}):
        values = list(row.children)
        name = values[1].find('span').contents[0]
        code = values[2].find('span').contents
        if len(code) > 0:
            icons.append(code[0])
    groups.append((group_name, icons))
       

In [16]:
csv_path = glob.glob('./Export-*/SF Symbols *.csv')[0]
notion_path = glob.glob('./Export-*/')[0]

symbols = pd.read_csv(csv_path)
symbols = symbols.astype({'Alias': str})
symbols['Alias'] = symbols['Alias'].replace('nan', '')
symbols['Free'] = symbols['Free'] == 'Yes'
symbols.head()
symbols[symbols['Deprecated'].notna()].head()

,Symbol,Name,Code,Status,Category,Free,File,Features,Alias,Source,Source Link,Deprecated,Availability,Property
30,SF%20Symbols%20Library%201c95183b9f974adabf307...,Soccer,backport.soccerball,Released,Activities,True,SF%20Symbols%20Library%201c95183b9f974adabf307...,"Hierarchical, Weights","Football, Soccer, Basketball",Apple SF Symbols,NaN,custom.ball,iOS 16,NaN
34,SF%20Symbols%20Library%201c95183b9f974adabf307...,Paw,pawprint.fill,Released,Animals,True,NaN,Hierarchical,,Apple SF Symbols,NaN,custom.dog,iOS 14,NaN
35,SF%20Symbols%20Library%201c95183b9f974adabf307...,Dumbbell,backport.dumbbell.fill,Released,Activities,True,SF%20Symbols%20Library%201c95183b9f974adabf307...,Weights,,Apple SF Symbols,NaN,custom.dumbbell,iOS 16,NaN
36,SF%20Symbols%20Library%201c95183b9f974adabf307...,Shower,backport.shower.fill,Released,Bathroom,True,SF%20Symbols%20Library%201c95183b9f974adabf307...,NaN,,Apple SF Symbols,NaN,custom.shower,iOS 16,NaN
43,SF%20Symbols%20Library%201c95183b9f974adabf307...,Walking,figure.walk,Released,Transport,True,NaN,"Hierarchical, Weights",,Apple SF Symbols,NaN,custom.walk,iOS 14,NaN


In [17]:
categories_path = glob.glob('./Categories *.csv')[0]
categories = pd.read_csv(categories_path)
categories

,Name,Symbol,Identifier
0,Bathroom,backport.shower.fill,bathroom
1,Cloths,tshirt.fill,cloths
2,Household,house.fill,household
3,School,graduationcap.fill,school
4,Office,desktopcomputer,office
5,Transport,bicycle,transport
6,Activities,flame.fill,activities
7,Animals,pawprint.fill,animals
8,Human,person.fill,human
9,Food & Drinks,fork.knife,food


In [18]:
json_dict = list()
for group in groups:
    name = group[0]
    category = categories[categories['Name'] == name].iloc[0]
    icons = list()
    for icon_code in group[1]:
        icon_query = symbols[symbols['Code'] == icon_code]
        if len(icon_query) <= 0:
            continue
        icon = icon_query.iloc[0]
        alias = list([s for s in icon['Alias'].split(', ') if len(s) > 0])
        availability = int(icon['Availability'].split(' ')[1])
        icon_dict = {
            'code': icon_code,
            # 'name': icon['Name'],
            'availability': availability,
            'alias': alias,
            'isFree': bool(icon['Free'])
        }
        deprecated = icon['Deprecated']
        if pd.notna(deprecated):
            icon_dict['deprecated'] = deprecated

        icons.append(icon_dict)
    group_dict = {
        # 'name': name,
        'symbols': icons,
        'symbol': category['Symbol'],
        'identifier': f"symbol.category.{category['Identifier']}"
    }
    json_dict.append(group_dict)

json_string = json.dumps(json_dict)
with open("symbols.json", "w") as outfile:
    outfile.write(json_string)

In [19]:
released = symbols[symbols['Code'].notna()]
lines = list()
for i, row in released.iterrows():
    lines.append(f'"symbol.{row["Code"]}" = "{row["Name"]}";\n')

for i, row in categories.iterrows():
    lines.append(f'"symbol.category.{row["Identifier"]}" = "{row["Name"]}";\n')
    
with open("Symbols.strings", "w") as file:
    file.writelines(lines)


In [20]:
## Export Custom Symbols
asset_path = 'Symbols.xcassets'
if os.path.exists(asset_path):
    shutil.rmtree(asset_path)
os.mkdir(asset_path)
custom_path = "CustomSymbols"
if os.path.exists(custom_path):
    shutil.rmtree(custom_path)
os.mkdir(custom_path)
for _, symbol in symbols[~symbols['File'].isnull()].iterrows():
    file = str.replace(symbol['File'], '%20', ' ')
    print(file)
    code = symbol['Code']
    set_path = f'{asset_path}/{code}.symbolset'
    os.mkdir(set_path)
    shutil.copyfile(os.path.join(notion_path,file), f'{set_path}/{code}.svg')
    shutil.copyfile(os.path.join(notion_path,file), os.path.join(custom_path, f'{code}.svg'))
    print()
    with open('Contents-template.json', "r") as template:
        contents = json.load(template)
    contents['symbols'][0]['filename'] = f'{code}.svg'
    with open(os.path.join(set_path, 'Contents.json'), 'w') as f:
        json.dump(contents, f, indent=4)

SF Symbols Library 1c95183b9f974adabf30710c42dbeccb/custom.tree.svg

SF Symbols Library 1c95183b9f974adabf30710c42dbeccb/custom.dog.svg

SF Symbols Library 1c95183b9f974adabf30710c42dbeccb/custom.soap.svg

SF Symbols Library 1c95183b9f974adabf30710c42dbeccb/custom.hanger.svg

SF Symbols Library 1c95183b9f974adabf30710c42dbeccb/custom.water_glass.svg

SF Symbols Library 1c95183b9f974adabf30710c42dbeccb/custom.road.svg

SF Symbols Library 1c95183b9f974adabf30710c42dbeccb/custom.toilet_paper.svg

SF Symbols Library 1c95183b9f974adabf30710c42dbeccb/window.casement.svg

SF Symbols Library 1c95183b9f974adabf30710c42dbeccb/stove.fill.svg

SF Symbols Library 1c95183b9f974adabf30710c42dbeccb/toilet.fill.svg

SF Symbols Library 1c95183b9f974adabf30710c42dbeccb/figure.dance.svg

SF Symbols Library 1c95183b9f974adabf30710c42dbeccb/custom.horse.svg

SF Symbols Library 1c95183b9f974adabf30710c42dbeccb/figure.martial.arts.svg

SF Symbols Library 1c95183b9f974adabf30710c42dbeccb/figure.open.water.swim

In [21]:
released = symbols[pd.notnull(symbols['Code'])]

print(len(released))
# released.head()

learn = list()

icons = list()
shutil.rmtree('test_data')
os.mkdir("test_data")
for i, row in released.iterrows(): 
    examples = list([s for s in row['Alias'].split(', ') if len(s) > 0])
    print(row['Name'], examples)
    icons.append({'symbol': row['Code'], 'title': row['Name']})
    path = os.path.join("test_data", row['Code'])
    if len(examples) > 0:
        os.mkdir(path)
    for index, title in enumerate(examples):
        learn.append(
            {
                'label': row['Code'],
                'text': title,
                'languages': ['en-US']
            }
        )
        with open(os.path.join(path, f"test_{index}.txt"), 'w') as f:
            f.write(title)

lean_export = pd.DataFrame(learn)

lean_export.to_json('predefined_icons_learn.json', orient="records")
lean_export[["label", "text"]].to_csv("predefined_icons_learn.csv", index=False)


icons_export = pd.DataFrame(icons)

icons_export.to_json('icons.json', orient="records")


536
Tree []
Dog []
Pencil []
Soap []
Cloths Hanger []
Water Glass []
Road []
Toilet Paper []
Windows []
Cooking Stove []
Toilet []
Dancing []
Horse []
Martial Arts []
Swimming []
Tent []
Laundry Basket []
Apple []
Frying Pan []
Candy []
Shovel []
Ax []
Flowers []
Plant []
Frog []
Knitting []
Baby Bottle []
Folding Cloths []
Hockey []
Reminders ['Apple Reminders', 'Task', 'Todo']
Soccer ['Football', 'Soccer', 'Basketball']
Cat []
Chemistry []
Cooking []
Paw []
Dumbbell []
Shower []
Toothbrush []
Hairdryer []
Language []
Lipstick []
Razor []
Sitting []
Walking []
Running []
School []
Subway []
Ticket []
Coffee []
Meal []
Party []
Airplane []
Arrival []
Departure []
Alcohol []
Alarm []
Ant []
Box []
Recycling []
Merge []
Email []
Physics []
Bag []
Bandage []
Battery []
Bell []
Bike []
Birthday ['birthday cake']
Baby Carriage []
Bitcoin []
Electric Car []
Bolt []
Book []
Bookmark []
Briefcase []
Bubble []
Danger []
Bus []
Couch []
Magic []
Unlock []
Hourglass []
Newspaper []
Wrench []
Phot

In [22]:

def delete_symbol(code):
    path = 'SystemSymbols/' + code + '@2x.png'
    if os.path.exists(path):
        # print("Deleted: ", code)
        os.remove(path)
system = symbols[symbols['Source'] == 'Apple SF Symbols']
for i, row in system.iterrows(): 
    code = row['Code'].replace('backport.', '')
    non_filled = code.replace('.fill', '')
    filled = code + '.fill'
    delete_symbol(code)
    delete_symbol(non_filled)
    delete_symbol(filled)
    delete_symbol(filled + '.circle')
    delete_symbol(non_filled + '.circle')
    delete_symbol(non_filled + '.circle.fill')
